#### Лабораторная работа № 2. Разработка нейросетевых функций. Операция Convolution 3D
#### БВТ2003 Никитин Степан Романович

In [89]:
import torch
import numpy as np
from typing import Union

In [128]:
class Conv3D:
    def __init__(self, in_channels: int, out_channels: int, kernel_size: Union[int, tuple], stride: Union[int, tuple] = 1, groups: int = 1) -> None:
        """
        Конструктор класса Conv3D
        """
        self.in_channels = in_channels     # количество входных каналов (каналов входного тензора)
        self.out_channels = out_channels   # количество выходных каналов (каналов результата свертки)
        self.kernel_size = kernel_size     # размер ядра свертки (предполагается квадратное ядро)
        self.stride = stride               # шаг свертки
        self.groups = groups               # количество групп для разделения входных и выходных каналов

        # преобразование kernel_size в кортеж, если он целое число
        if isinstance(kernel_size, int):
            kernel_size = (kernel_size, kernel_size, kernel_size)

        # инициализация весов (ядер свертки)
        self.weights = np.random.randn(out_channels, in_channels // groups, *kernel_size)
        self.bias = np.zeros((out_channels,))

    def forward(self, input_tensor: np.ndarray) -> np.ndarray:
        """
        Метод для выполнения трехмерной операции свертки
        """
        batch_size, in_channels, input_depth, input_height, input_width = input_tensor.shape
        kernel_size = self.kernel_size

        # преобразование kernel_size в кортеж, если он целое число
        if isinstance(kernel_size, int):
            kernel_size = (kernel_size, kernel_size, kernel_size)

        # проверка на корректные размеры входного тензора
        if input_depth < kernel_size[0] or input_height < kernel_size[1] or input_width < kernel_size[2]:
            raise ValueError("Неверный размер ядра для входного тензора")

        # вычисление размеров результата свертки
        if isinstance(self.stride, int):
            stride_d = stride_h = stride_w = self.stride
        elif isinstance(self.stride, tuple) and len(self.stride) == 3:
            stride_d, stride_h, stride_w = self.stride
        else:
            raise ValueError("Недопустимый формат шага. Должно быть целое число или кортеж из трех целых чисел")

        output_depth = (input_depth - kernel_size[0]) // stride_d + 1
        output_height = (input_height - kernel_size[1]) // stride_h + 1
        output_width = (input_width - kernel_size[2]) // stride_w + 1

        # инициализация результата свертки
        result = np.zeros((self.out_channels, output_depth, output_height, output_width))

        # выполнение свертки
        for d in range(0, input_depth - kernel_size[0] + 1, stride_d):
            for i in range(0, input_height - kernel_size[1] + 1, stride_h):
                for j in range(0, input_width - kernel_size[2] + 1, stride_w):
                    # окно свертки
                    window = input_tensor[:, :, d:d + kernel_size[0], i:i + kernel_size[1], j:j + kernel_size[2]]

                    # вычисление свертки и добавление bias
                    result[:, d // stride_d, i // stride_h, j // stride_w] = np.sum(window * self.weights, axis=(1, 2, 3, 4)) + self.bias

        return result

In [134]:
def run_3d_test(in_channels: int, out_channels: int, kernel_size: Union[int, tuple], stride: Union[int, tuple],
                padding: Union[int, tuple], dilation: int, groups: int, bias: bool, padding_mode: str) -> None:
    """
    Функция для выполнения теста трехмерной операции свертки
    """
    # создание экземпляра Conv3D
    conv3d_layer = Conv3D(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size, stride=stride)

    # создание трехмерного тензора для теста
    input_tensor_3d = np.random.randn(1, in_channels, 10, 10, 10)

    # выполнение трехмерной свертки
    result_3d = conv3d_layer.forward(input_tensor_3d)

    # преобразование результатов в формат torch.Tensor для сравнения с PyTorch
    output_tensor_torch = torch.nn.functional.conv3d(torch.tensor(input_tensor_3d), 
                                                      torch.tensor(conv3d_layer.weights), 
                                                      bias=torch.tensor(conv3d_layer.bias), 
                                                      stride=conv3d_layer.stride, 
                                                      padding=padding,
                                                      dilation=dilation,
                                                      groups=1).numpy()

    # проверка совпадения результатов
    are_results_equal = np.allclose(result_3d, output_tensor_torch)
    
    print(f"Тест с параметрами:")
    print(f"  - in_channels: {in_channels}")
    print(f"  - out_channels: {out_channels}")
    print(f"  - kernel_size: {kernel_size}")
    print(f"  - stride: {stride}")
    print(f"  - padding: {padding}")
    print(f"  - dilation: {dilation}")
    print(f"  - groups: {groups}")
    print(f"  - bias: {bias}")
    print(f"  - padding_mode: {padding_mode}")
    print(f"\nСовпадение результатов: {are_results_equal}")
    print(f"\n{'-'*75}\n")

In [141]:
def test_3d_params() -> None:
    """
    Функция для тестирования различных параметров трехмерной операции свертки
    """
    # параметры тестирования для первой группы
    run_3d_test(in_channels=8, out_channels=20, kernel_size=3, stride=1, padding=0, dilation=1, groups=2, bias=True, padding_mode='zeros')
    
    # параметры тестирования для дополнительной группы
    run_3d_test(in_channels=3, out_channels=2, kernel_size=3, stride=(2, 2, 2), padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros')
    
    # параметры тестирования для дополнительной группы
    run_3d_test(in_channels=8, out_channels=20, kernel_size=3, stride=1, padding=0, dilation=1, groups=2, bias=True, padding_mode='zeros')

In [142]:
# параметры тестирования для дополнительной группы
input_tensor_3d = np.random.randn(1, 3, 5, 5, 5)
conv3d_layer = Conv3D(in_channels=3, out_channels=2, kernel_size=3, stride=(2, 2, 2))
output_tensor_3d = conv3d_layer.forward(input_tensor_3d)

print(f"Результат свертки (Conv3D):\n {output_tensor_3d}")

Результат свертки (Conv3D):
 [[[[  5.00813705  -5.3570865 ]
   [ -9.5868979  -10.91819192]]

  [[  8.51395494  15.32848853]
   [  6.67337      9.3429326 ]]]


 [[[  2.97462961  -4.41357134]
   [  4.83494784  -6.57586784]]

  [[ -2.73776324 -10.60930908]
   [  4.43325231  -2.49800322]]]]


In [143]:
# преобразование результатов в формат torch.Tensor для сравнения с PyTorch
input_tensor_torch_3d = torch.tensor(input_tensor_3d)
output_tensor_torch_3d = torch.nn.functional.conv3d(input_tensor_torch_3d,
                                                     torch.tensor(conv3d_layer.weights),
                                                     bias=torch.tensor(conv3d_layer.bias),
                                                     stride=(2, 2, 2)).numpy()

print(f"Результат свертки (PyTorch):\n {output_tensor_torch_3d}")

Результат свертки (PyTorch):
 [[[[[  5.00813705  -5.3570865 ]
    [ -9.5868979  -10.91819192]]

   [[  8.51395494  15.32848853]
    [  6.67337      9.3429326 ]]]


  [[[  2.97462961  -4.41357134]
    [  4.83494784  -6.57586784]]

   [[ -2.73776324 -10.60930908]
    [  4.43325231  -2.49800322]]]]]


In [144]:
test_3d_params()

Тест с параметрами:
  - in_channels: 8
  - out_channels: 20
  - kernel_size: 3
  - stride: 1
  - padding: 0
  - dilation: 1
  - groups: 2
  - bias: True
  - padding_mode: zeros

Совпадение результатов: True

---------------------------------------------------------------------------

Тест с параметрами:
  - in_channels: 3
  - out_channels: 2
  - kernel_size: 3
  - stride: (2, 2, 2)
  - padding: 0
  - dilation: 1
  - groups: 1
  - bias: True
  - padding_mode: zeros

Совпадение результатов: True

---------------------------------------------------------------------------

Тест с параметрами:
  - in_channels: 8
  - out_channels: 20
  - kernel_size: 3
  - stride: 1
  - padding: 0
  - dilation: 1
  - groups: 2
  - bias: True
  - padding_mode: zeros

Совпадение результатов: True

---------------------------------------------------------------------------

